## Problem 1
Perform an appropriate train-test split on the data. In your report, justify why <br>
you chose to do the split in the way you did. In further problems, only ever <br>
perform exploratory data analysis on the training data, as testing data should <br>
never be used to draw hypotheses.

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

RANDOM_STATE = 1

raw_df = pd.read_csv("life_expectancy.csv")
# Test size of 0.25 because this was recommended in a lecture.
test_df, train_df = train_test_split(raw_df, test_size=0.25, random_state=RANDOM_STATE)
print(train_df)


                               Country  Year  \
850                             Canada  2002   
689                             Brazil  2013   
5301  Saint Vincent and the Grenadines  2006   
2093                            Guyana  2015   
3253                          Maldives  2005   
...                                ...   ...   
3645                          Malaysia  2020   
2423                           Iceland  1998   
854                             Canada  2006   
2981                     Liechtenstein  2007   
4577             Sao Tome and Principe  1996   

      Mean Years of Schooling, female (years)  \
850                                 12.755616   
689                                  7.475190   
5301                                      NaN   
2093                                 8.421454   
3253                                 2.660000   
...                                       ...   
3645                                10.568640   
2423                           